In [ ]:
!pip install pandas
!pip install pandas openpyxl
!pip install xlrd


In [ ]:
import pandas as pd 
import os
import numpy as np

In [ ]:
def answer_one():
    file_name= os.path.join('Energy Indicators.xls')
    df = pd.read_excel(file_name, skiprows=17, skipfooter=38)
    pd.set_option('display.max_rows', None)  
    pd.set_option('display.max_columns', None) 
    energy = pd.DataFrame(df)
    energy

    del energy['Unnamed: 0']
    del energy['Unnamed: 1']

    energy.rename(columns = {'Unnamed: 2':'Country', 'Petajoules':'Energy Supply', 'Gigajoules':'Energy Supply per Capita', '%':'% Renewable'}, inplace = True) 
    
    def convert_petajoule_to_gigajoules(x):
        if x == '...':
            return x
        else:
            return x * 10**6

    energy['Energy Supply'] = energy['Energy Supply'].apply(convert_petajoule_to_gigajoules)

    energy[['Energy Supply', 'Energy Supply per Capita']] = np.where(
    energy[['Energy Supply', 'Energy Supply per Capita']] == '...',
    np.nan,
    energy[['Energy Supply', 'Energy Supply per Capita']]
    )

    def delete_number(x):
        return ' '.join([char for char in str(x) if not char.isdigit()])

    energy['Country'] = energy['Country'].apply(lambda x: ''.join([char for char in str(x) if not char.isdigit()]))

    def remove_text_in_brackets(input_string):
        result = []
        inside_brackets = False
    
        for i, char in enumerate(input_string):
            if char == '(':
                inside_brackets = True
            elif char == ')':
                inside_brackets = False
            elif not inside_brackets:
                result.append(char)
    
        return ''.join(result).strip()

    energy['Country'] = energy['Country'].apply(remove_text_in_brackets)

    dict_one = {'Republic of Korea': 'South Korea', 'United States of America': 'United States', 
        'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom', 'China, Hong Kong Special Administrative Region': 'Hong Kong'}

    for key, var in dict_one.items():
        energy['Country'] = energy['Country'].replace(key, var)

    file_name_csv = 'world_bank.csv'
    df_two = pd.read_csv(file_name_csv, sep = ',', header= None, skiprows=4, encoding='utf-8')
    GDP = pd.DataFrame(df_two)

    dict_two = {'Korea, Rep.': 'South Korea', 'Iran, Islamic Rep.': 'Iran', 
        'Hong Kong SAR, China': 'Hong Kong'}

    #for key, var in dict_two.items():
    #    GDP['Country Name'] = GDP['Country Name'].replace(key, var)
    GDP[0] = GDP[0].replace(dict_two)
    GDP.columns = GDP.iloc[0]  
    GDP = GDP[1:]
    GDP.rename(columns = {'Country Name':'Country'}, inplace = True) 
    GDP['Country'] = GDP['Country'].apply(remove_text_in_brackets)

    file_name_three= os.path.join('scimagojr.xlsx')
    df_three = pd.read_excel(file_name_three, skiprows=0, skipfooter=0)
    pd.set_option('display.max_rows', None)  
    pd.set_option('display.max_columns', None) 
    ScimEn = pd.DataFrame(df_three)
    ScimEn['Country'] = ScimEn['Country'].apply(remove_text_in_brackets)

    energy.set_index('Country', inplace=True)
    GDP.set_index('Country', inplace=True)
    ScimEn.set_index('Country', inplace=True)

    old_column_value = 1960.0
    for i in range(1960, 2016, 1):
        GDP.rename(columns={old_column_value: int(old_column_value)}, inplace=True)
        old_column_value += 1

    GDP_selected = GDP.loc[:, 2006:2015]
    df_four = energy.merge(ScimEn, on="Country", how="inner")
    df_four = df_four.merge(GDP_selected, on="Country", how="inner")
    df_four = df_four.drop(columns=['Region'])
    df_filtered = df_four[df_four['Rank'] <= 15]
    df_filtered = df_filtered.sort_values(by='Rank')
        
    return df_filtered

In [ ]:
question_1 = answer_one()
question_1

In [ ]:
def answer_two():
    Top15 = answer_one()
    for col in Top15.columns:
        if col == 2006:
            break
        del Top15[col]

    Top15['row_mean'] = Top15.mean(axis=1)
    avgGDP = Top15['row_mean'].sort_values(ascending=False)
    return avgGDP

In [ ]:
question_2 = answer_two()
question_2

In [ ]:
def answer_three():
    Top15 = answer_one()
    for col in Top15.columns:
        if col == 2006:
            break
        del Top15[col]

    Top15['row_mean'] = Top15.mean(axis=1)

    top_6_country = Top15['row_mean'].sort_values(ascending=False).index[5]
    gdp_2006 = Top15.loc[top_6_country, 2006]
    gdp_2015 = Top15.loc[top_6_country, 2015]
    percent_change = ((gdp_2015 - gdp_2006) / gdp_2006) * 100
    
    return float(percent_change)


In [ ]:
question_3 = answer_three()
question_3

In [ ]:
def answer_four():
    Top15 = answer_one()
    Top15['Self-citation Ratio'] = Top15['Self-citations'] / Top15['Citations']
    max_ratio = Top15['Self-citation Ratio'].max()
    country_with_max_ratio = Top15['Self-citation Ratio'].idxmax()
    return (country_with_max_ratio, float(max_ratio))

In [ ]:
question_4 = answer_four()
question_4

In [ ]:
def answer_five():
    Top15 = answer_one()
    Top15['Population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita'] 
    third_population_country = Top15.sort_values(by='Population', ascending=False).index[2]
    return str(third_population_country)

In [ ]:
question_5 = answer_five()
question_5